In [2]:
# New data로 test
from PIL import Image
import torch
import torch.nn as nn

def img_to_tensor(dir):
    new_img = Image.open(dir)
    new_img = new_img.resize((600, 600))
    print(new_img.size, new_img.mode)

new_pork = img_to_tensor('../DATA/pork3.jpg')
new_beef = img_to_tensor('../DATA/beef.jpg')

(600, 600) RGB
(600, 600) RGB


In [8]:
from torchvision import transforms
import torch.nn.functional as F

class MeatModel(nn.Module):
    def __init__(self):
        super(MeatModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)  # (3, 600, 600) -> (32, 600, 600), 가로세로는 일정
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)  # (32, 600, 600) -> (64, 600, 600)
        self.pool = nn.MaxPool2d(kernel_size=2)  # (64, 600, 600) -> (64, 300, 300) -> (64, 150, 150)
        self.fc1 = nn.Linear(64 * 150 * 150, 128)
        self.fc2 = nn.Linear(128, 2)    # 0, 1이면 2개의 클래스, 1인가 2인가?
        
    def forward(self, x):
        y = F.relu(self.conv1(x))   # (3, 600, 600) -> (32, 600, 600)
        y = self.pool(y)            # (32, 600, 600) -> (32, 300, 300)
        y = F.relu(self.conv2(y))   # (32, 300, 300) -> (64, 300, 300)
        y = self.pool(y)            # (64, 300, 300) -> (64, 150, 150)
        
        y = y.view(-1, 64 * 150 * 150)  # 1차원으로 펼치기 : ?? 근데 왜 300이지? -> 150 맞다! fc1을 300으로 해서 오류
        y = F.relu(self.fc1(y))    # (64 * 150 * 150) -> (128)
        y = self.fc2(y)            # (128) -> (2)
        return y

In [9]:
# load model from pickle
import pickle
import torch

model = torch.load('meat_model.pkl')

RuntimeError: Invalid magic number; corrupt file?

In [10]:
# New data로 test

model.eval()
pred = model(new_pork)
print(pred)

NameError: name 'model' is not defined

In [ ]:
# ㅠㅠ 로드 실패로 new data 확인 불가
# 대신 삼겹살은 성공